In [1]:
RUBY_CMD = 'ruby'
BUNDLE_CMD = 'bundle exec ruby'
META_CLI = File.absolute_path('/opt/OpenStudio-server/bin/openstudio_meta')
PROJECT = File.absolute_path('/opt/OpenStudio-server/server/spec/files')
HOST = '192.168.65.3'

"192.168.65.3"

In [2]:
@host = HOST
@project = PROJECT
@meta_cli = META_CLI
@ruby_cmd = RUBY_CMD
@bundle_cmd = BUNDLE_CMD

"bundle exec ruby"

In [3]:
#URBANopt run

In [4]:
command = "#{@meta_cli} run_analysis --debug --verbose '#{@project}/URBANopt_template.json' 'http://#{@host}' -z 'URBANopt_071' -a single_run"

"/opt/OpenStudio-server/bin/openstudio_meta run_analysis --debug --verbose '/opt/OpenStudio-server/server/spec/files/URBANopt_template.json' 'http://192.168.65.3' -z 'URBANopt_071' -a single_run"

In [5]:
run_analysis = system(command)

true

In [6]:
require 'rest-client'
require 'json'
a = RestClient.get "http://#{@host}/analyses.json"
a = JSON.parse(a, symbolize_names: true)
a = a.sort { |x, y| x[:created_at] <=> y[:created_at] }.reverse
analysis = a[0]
analysis_id = analysis[:_id]

"f03c2954-6d7c-4c49-872a-98f77fc5cc47"

In [7]:
a = RestClient.get "http://#{@host}/analyses/#{analysis_id}/status.json"
a = JSON.parse(a, symbolize_names: true)
status = a[:analysis][:status]

"queued"

In [8]:
a = RestClient.get "http://#{@host}/analyses/#{analysis_id}.json"
a = JSON.parse(a, symbolize_names: true)
status = a[:analysis][:problem][:analysis_type]

"single_run"

In [9]:
a[:analysis][:urbanopt]

true

In [11]:
# get all data points in this analysis
a = RestClient.get "http://#{@host}/data_points.json"
a = JSON.parse(a, symbolize_names: true)
data_points = []
a.each do |data_point|
  if data_point[:analysis_id] == analysis_id
    data_points << data_point
  end
end
data_points.size

1

In [12]:
data_point_id = data_points[0][:_id]

"2ba8093e-5486-483d-bd48-806af3571140"

In [13]:
dp = RestClient.get "http://#{@host}/data_points/#{data_point_id}.json"
dp = JSON.parse(dp, symbolize_names: true)
data_points_status = dp[:data_point][:status]

"started"

In [14]:
dp[:data_point][:status_message]

""

In [48]:
results = dp[:data_point][:results]

{}

In [49]:
zip = RestClient.get "http://#{@host}/data_points/#{data_point_id}/download_result_file?filename=data_point.zip"

<RestClient::Response 200 "PK\x03\x04\x14\x00\x00\x00\x00\x00\xB4...">

In [50]:
zip.headers

{:content_type=>"application/zip", :transfer_encoding=>"chunked", :connection=>"close", :status=>"200 OK", :cache_control=>"private", :vary=>"Accept", :referrer_policy=>"strict-origin-when-cross-origin", :x_permitted_cross_domain_policies=>"none", :x_xss_protection=>"1; mode=block", :x_request_id=>"c5d0c559-bb2e-45fc-879a-525bbc8782e8", :content_disposition=>"attachment; filename=\"data_point.zip\"; filename*=UTF-8''data_point.zip", :x_download_options=>"noopen", :content_transfer_encoding=>"binary", :etag=>"W/\"11d9663d4426e5eef626a3caebc7d70a\"", :x_runtime=>"0.158127", :x_content_type_options=>"nosniff", :date=>"Wed, 22 Jun 2022 17:09:54 GMT", :x_powered_by=>"Phusion Passenger 6.0.2", :server=>"nginx/1.15.8 + Phusion Passenger 6.0.2"}

In [54]:
zip.headers[:content_disposition]

"attachment; filename=\"data_point.zip\"; filename*=UTF-8''data_point.zip"

In [57]:
require 'open-uri'
File.write 'datapoint.zip', URI.open("http://#{@host}/data_points/#{data_point_id}/download_result_file?filename=data_point.zip").read

163118268